<a href="https://colab.research.google.com/github/Swaleh-developer/CRUD-django-task/blob/main/Swaleh_Dphi_DL_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Download Images
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1f7uslI-ZHidriQFZR966_aILjlkgDN76', dest_path='/content/eye_gender_data.zip', unzip=True)

In [2]:
#Loading libraries
import pandas as pd # Data analysis and manipultion tool
import numpy as np # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf # Deep Learning Tool
import os # OS module in Python provides a way of using operating system dependent functionality
import cv2 # Library for image processing
from sklearn.model_selection import train_test_split # For splitting the data into train and validation set
from sklearn.metrics import f1_score

In [5]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving eye_gender_data.zip to eye_gender_data.zip
User uploaded file "eye_gender_data.zip" with length 26794203 bytes


In [3]:
!unzip eye_gender_data.zip


Streaming output truncated to the last 5000 lines.
  inflating: eye_gender_data/train/Image_4799.jpg  
  inflating: eye_gender_data/train/Image_48.jpg  
  inflating: eye_gender_data/train/Image_480.jpg  
  inflating: eye_gender_data/train/Image_4800.jpg  
  inflating: eye_gender_data/train/Image_4801.jpg  
  inflating: eye_gender_data/train/Image_4802.jpg  
  inflating: eye_gender_data/train/Image_4803.jpg  
  inflating: eye_gender_data/train/Image_4804.jpg  
  inflating: eye_gender_data/train/Image_4805.jpg  
  inflating: eye_gender_data/train/Image_4806.jpg  
  inflating: eye_gender_data/train/Image_4807.jpg  
  inflating: eye_gender_data/train/Image_4808.jpg  
  inflating: eye_gender_data/train/Image_4809.jpg  
  inflating: eye_gender_data/train/Image_481.jpg  
  inflating: eye_gender_data/train/Image_4810.jpg  
  inflating: eye_gender_data/train/Image_4811.jpg  
  inflating: eye_gender_data/train/Image_4812.jpg  
  inflating: eye_gender_data/train/Image_4813.jpg  
  inflating: eye_

In [4]:
#Loading and Preparing Training Data

labels = pd.read_csv("/content/eye_gender_data/Training_set.csv") # loading the labels
file_paths = [[fname, '/content/eye_gender_data/train/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')

# Confirm if number of images is same as number of labels given
if len(labels) == len(file_paths):
  print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
  print('Number of labels does not match the number of filenames')


train_data

Number of labels i.e.  9220 matches the number of filenames i.e.  9220


,filename,filepaths,label
0,Image_1.jpg,/content/eye_gender_data/train/Image_1.jpg,male
1,Image_2.jpg,/content/eye_gender_data/train/Image_2.jpg,female
2,Image_3.jpg,/content/eye_gender_data/train/Image_3.jpg,female
3,Image_4.jpg,/content/eye_gender_data/train/Image_4.jpg,female
4,Image_5.jpg,/content/eye_gender_data/train/Image_5.jpg,male
...,...,...,...
9215,Image_9216.jpg,/content/eye_gender_data/train/Image_9216.jpg,male
9216,Image_9217.jpg,/content/eye_gender_data/train/Image_9217.jpg,male
9217,Image_9218.jpg,/content/eye_gender_data/train/Image_9218.jpg,male
9218,Image_9219.jpg,/content/eye_gender_data/train/Image_9219.jpg,male


In [5]:
#Converting categorical data to numerical data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])
train_data['label'].shape

#Train data is ready


(9220,)

In [6]:
#Data Pre-processing
"""
It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values
"""
data = [] # initialize an empty numpy array
image_size = 100 # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale
  print(train_data['filepaths'][i])
  new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array
  data.append([new_img_array, train_data['label'][i]])




Streaming output truncated to the last 5000 lines.
/content/eye_gender_data/train/Image_4221.jpg
/content/eye_gender_data/train/Image_4222.jpg
/content/eye_gender_data/train/Image_4223.jpg
/content/eye_gender_data/train/Image_4224.jpg
/content/eye_gender_data/train/Image_4225.jpg
/content/eye_gender_data/train/Image_4226.jpg
/content/eye_gender_data/train/Image_4227.jpg
/content/eye_gender_data/train/Image_4228.jpg
/content/eye_gender_data/train/Image_4229.jpg
/content/eye_gender_data/train/Image_4230.jpg
/content/eye_gender_data/train/Image_4231.jpg
/content/eye_gender_data/train/Image_4232.jpg
/content/eye_gender_data/train/Image_4233.jpg
/content/eye_gender_data/train/Image_4234.jpg
/content/eye_gender_data/train/Image_4235.jpg
/content/eye_gender_data/train/Image_4236.jpg
/content/eye_gender_data/train/Image_4237.jpg
/content/eye_gender_data/train/Image_4238.jpg
/content/eye_gender_data/train/Image_4239.jpg
/content/eye_gender_data/train/Image_4240.jpg
/content/eye_gender_data/trai

In [7]:
#pixels of image
data[10]


[array([[ 95, 102, 114, ..., 201, 202, 202],
        [ 99, 106, 118, ..., 197, 197, 197],
        [105, 112, 124, ..., 189, 189, 189],
        ...,
        [181, 181, 182, ..., 189, 188, 187],
        [182, 182, 182, ..., 187, 186, 184],
        [183, 183, 183, ..., 186, 184, 183]], dtype=uint8), 0]

In [8]:
np.random.shuffle(data)


In [9]:
#Images and labels separation

x = []
y = []
for image in data:
    x.append(image[0])
    y.append(image[1])

# converting x & y to numpy array as they are list

y = np.array(y)

x = np.array(x)

x=x/255

x

array([[[0.65882353, 0.67843137, 0.70980392, ..., 0.56078431,
         0.62745098, 0.6745098 ],
        [0.6627451 , 0.68235294, 0.71372549, ..., 0.56862745,
         0.63137255, 0.67058824],
        [0.67058824, 0.69019608, 0.72156863, ..., 0.58431373,
         0.63529412, 0.67058824],
        ...,
        [0.8       , 0.80392157, 0.80784314, ..., 0.83529412,
         0.83529412, 0.83529412],
        [0.79607843, 0.8       , 0.80392157, ..., 0.83137255,
         0.83137255, 0.83137255],
        [0.79607843, 0.8       , 0.80392157, ..., 0.82745098,
         0.82745098, 0.82745098]],

       [[0.42352941, 0.42745098, 0.43137255, ..., 0.62352941,
         0.63921569, 0.65098039],
        [0.41960784, 0.41960784, 0.42352941, ..., 0.60392157,
         0.62745098, 0.64705882],
        [0.41176471, 0.41176471, 0.41176471, ..., 0.57647059,
         0.60784314, 0.63529412],
        ...,
        [0.48235294, 0.48627451, 0.49803922, ..., 0.58039216,
         0.57254902, 0.56862745],
        [0.4

In [10]:
np.unique(y, return_counts=True)


(array([0, 1]), array([4162, 5058]))

In [11]:
x = x.reshape(-1, 100, 100, 1)

In [12]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)

In [13]:
#Model Building and Hyperparameter Tuning
model = tf.keras.models.Sequential([
                                  tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu', input_shape=(100,100,1)),
                                  tf.keras.layers.MaxPooling2D((2,2)),
                                  tf.keras.layers.Dropout(0.25),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
                                  tf.keras.layers.MaxPooling2D((2,2)),
                                  tf.keras.layers.Dropout(0.25),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Flatten(),
                                  tf.keras.layers.Dense(512, activation='relu'),
                                  tf.keras.layers.Dropout(0.25),
                                  tf.keras.layers.Dense(128, activation='relu'),
                                  tf.keras.layers.Dropout(0.25),
                                  tf.keras.layers.Dense(64, activation='relu'),
                                  tf.keras.layers.Dense(1, activation='sigmoid')
])



In [14]:
# compile model
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# view model layers
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 256)       2560      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 256)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 49, 49, 256)       0         
                                                                 
 batch_normalization (BatchN  (None, 49, 49, 256)      1024      
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 128)       295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 128)      0

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

hist = model.fit(X_train, y_train, epochs=50, batch_size=250, callbacks=[callback])


Epoch 1/50
26/26 [==============================] - 895s 34s/step - loss: 1.1833 - accuracy: 0.6108
Epoch 2/50
26/26 [==============================] - 894s 34s/step - loss: 0.6143 - accuracy: 0.7233
Epoch 3/50
26/26 [==============================] - 875s 34s/step - loss: 0.5094 - accuracy: 0.7693
Epoch 4/50
26/26 [==============================] - 881s 34s/step - loss: 0.4461 - accuracy: 0.8028
Epoch 5/50
26/26 [==============================] - 935s 36s/step - loss: 0.4055 - accuracy: 0.8215
Epoch 6/50
26/26 [==============================] - 867s 33s/step - loss: 0.3696 - accuracy: 0.8403
Epoch 7/50
26/26 [==============================] - 852s 33s/step - loss: 0.3298 - accuracy: 0.8606
Epoch 8/50
26/26 [==============================] - 863s 33s/step - loss: 0.3144 - accuracy: 0.8685
Epoch 9/50
26/26 [==============================] - 864s 33s/step - loss: 0.2862 - accuracy: 0.8782
Epoch 10/50
26/26 [==============================] - 852s 33s/step - loss: 0.2669 - accuracy: 0.8818

In [18]:

model.evaluate(X_val,y_val)


87/87 [==============================] - 85s 980ms/step - loss: 1.4739 - accuracy: 0.6761


[1.473933458328247, 0.676066517829895]

In [19]:
#Testing data set
#Loading test data set

labels = pd.read_csv("/content/eye_gender_data/Testing_set.csv") # loading the labels
file_paths = [[fname, '/content/eye_gender_data/test/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_data = pd.merge(images, labels, how = 'inner', on = 'filename')

# Confirm if number of images is same as number of labels given
if len(labels) == len(file_paths):
  print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
  print('Number of labels does not match the number of filenames')

images.shape

Number of labels i.e.  2305 matches the number of filenames i.e.  2305


(2305, 2)

In [20]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.head()


,filename,filepaths
0,Image_1.jpg,/content/eye_gender_data/test/Image_1.jpg
1,Image_2.jpg,/content/eye_gender_data/test/Image_2.jpg
2,Image_3.jpg,/content/eye_gender_data/test/Image_3.jpg
3,Image_4.jpg,/content/eye_gender_data/test/Image_4.jpg
4,Image_5.jpg,/content/eye_gender_data/test/Image_5.jpg


In [22]:
test_pixel_data = [] # initialize an empty numpy array
image_size = 100 # image size taken is 100 here. one can take other size too
for i in range(len(test_images)):
  img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale
  new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array
  test_pixel_data.append(new_img_array)

test_pixel_data 

[array([[140, 141, 142, ..., 169, 171, 173],
        [138, 138, 138, ..., 166, 168, 170],
        [137, 135, 134, ..., 163, 164, 166],
        ...,
        [146, 150, 155, ..., 151, 151, 151],
        [146, 151, 157, ..., 148, 147, 146],
        [147, 152, 158, ..., 147, 145, 143]], dtype=uint8),
 array([[165, 168, 173, ...,  93,  89,  87],
        [160, 163, 169, ...,  89,  86,  84],
        [153, 156, 162, ...,  83,  80,  78],
        ...,
        [111, 113, 116, ..., 122, 120, 119],
        [111, 112, 115, ..., 125, 122, 121],
        [110, 112, 115, ..., 127, 124, 122]], dtype=uint8),
 array([[208, 206, 204, ..., 184, 186, 187],
        [207, 206, 204, ..., 182, 184, 185],
        [206, 206, 204, ..., 179, 181, 181],
        ...,
        [211, 212, 214, ..., 222, 222, 222],
        [210, 212, 214, ..., 221, 221, 221],
        [210, 211, 214, ..., 221, 221, 221]], dtype=uint8),
 array([[ 45,  43,  41, ..., 183, 180, 177],
        [ 35,  34,  32, ..., 181, 179, 177],
        [ 21,  2

In [23]:
test_pixel_data = np.array(test_pixel_data)/255

test_pixel_data = test_pixel_data.reshape(-1, 100, 100, 1)

In [24]:
pred = model.predict(test_pixel_data)

pred[:]

array([[0.99999845],
       [0.8131434 ],
       [1.        ],
       ...,
       [0.9999999 ],
       [0.99987006],
       [0.9999982 ]], dtype=float32)

In [25]:
prediction = []
for value in pred:
  prediction.append(np.argmax(value))



In [26]:
predictions = le.inverse_transform(prediction)

predictions

array(['female', 'female', 'female', ..., 'female', 'female', 'female'],
      dtype=object)

In [28]:
res = pd.DataFrame({'label': predictions}) 
res.to_csv("submission.csv", index = False)

# To download the csv file locally
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>